# Track A Computer Vision - Image Classification

## M0 : Quick Stard

### Confirm GPU is ready

In [1]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"

Sat Nov 15 09:14:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              2W /   50W |     231MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Point the notebook at the project folder

In [2]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "od-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone od-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\cv-project-starter\cv-project


### Install the project requirement

In [3]:
!pip install -r requirements.txt

### Téléchargement du dataset

In [4]:
from torchvision.datasets import OxfordIIITPet

ds = OxfordIIITPet(
    root="./data/pets",
    split="trainval",
    target_types="category",
    download=True
)

### Run the smoke test

In [5]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/cv_oxfordpet.yaml")
print(smoke_path.read_text())

{
  "loss": 4.0178351402282715,
  "batch_size": 32,
  "num_classes": 37,
  "device": "cuda"
}


# M1: Problem Scoping & Data Validation